# PINN Implementation of Ashourvan & Diamond Paper

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import grad

import mlflow
import mlflow.pytorch

from tqdm.notebook import tqdm

import imageio.v2 as imageio
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os

## Define Parameters

In [2]:
l = 1.0     # not provided by paper, need to check relation with Λ 
α = 6.0
D_c = 0.78
C_χ = 0.95
a_u = 1.0
μ_c = 0.78
β = 0.1
Λ = 4000.0
ϵ_c = 6.25

In [3]:
g_i = 5.1
ϵ_i = 0.002

In [4]:
physical_params = {
    'l': l,
    'α': α,
    'D_c': D_c,
    'C_χ': C_χ,
    'a_u': a_u,
    'μ_c': μ_c,
    'β': β,
    'Λ': Λ,
    'ϵ_c': ϵ_c,
    'g_i': g_i,
    'ϵ_i': ϵ_i
}

## Define PDEs

### Define repeated calculations

In [5]:
def compute_w(C_χ, l, ϵ, α, a_u, u):
    return C_χ * l**2 * ϵ / torch.sqrt(α**2 + a_u * u**2)

### Dynamic equation for mean density


In [6]:
def pde_mean_density(x, n_t, n_x, n_xx, ϵ, l, α, D_c):    
    intermediate = l**2 * ϵ * n_x / α
    intermediate_x = grad(intermediate, x, grad_outputs=torch.ones_like(intermediate), retain_graph=True, create_graph=True)[0]
    
    return (n_t - intermediate_x - D_c * n_xx)**2

### Dynamic equation for mean vorticity

In [7]:
def pde_mean_vorticity(x, ϵ, n_x, u_t, u_xx, l, α, μ_c, w):    
    intermediate = (l**2 * ϵ / α - w) * n_x
    intermediate_x = grad(intermediate, x, grad_outputs=torch.ones_like(intermediate), retain_graph=True, create_graph=True)[0]
    
    return (u_t - intermediate_x - w * u_xx - μ_c * u_xx)**2

### Dynamic equation for turbulent potential entrosphy

In [8]:
def pde_tpe(x, ϵ, n_x, u_x, ϵ_t, ϵ_x, l, β, Λ, ϵ_c, w):
    intermediate = l**2 * torch.sqrt(ϵ) * ϵ_x
    intermediate_x = grad(intermediate, x, grad_outputs=torch.ones_like(intermediate), retain_graph=True, create_graph=True)[0]
    
    return (ϵ_t - β * intermediate_x - Λ * (w * (n_x - u_x)**2 - ϵ**(3/2) / ϵ_c**0.5 + ϵ))**2

## Define Initial Conditions

In [9]:
def n_initial_cond(t, x):
    return -g_i * x

def u_initial_cond(t, x):
    return torch.zeros(x.shape, device=x.device.type)

def ϵ_initial_cond(t, x):
    return torch.full(x.shape, ϵ_i, device=x.device.type)

## Define Boundary Conditions

In [10]:
def n_boundary_cond(t, x):
    out = torch.full(x.shape, -g_i, device=x.device.type)
    out = out * x
    
    return out

def u_boundary_cond(t, x):
    return torch.zeros(x.shape, device=x.device.type)


def ϵ_x_boundary_cond(t, x):
    return torch.zeros(x.shape, device=x.device.type)

## PINN Implementation

In [11]:
class CustomOutputLayer(nn.Module):
    def __init__(self, pars):
        super(CustomOutputLayer, self).__init__()
        self.n = nn.Sequential(nn.Linear(pars['width'], 1))
        self.u = nn.Sequential(nn.Linear(pars['width'], 1))
        self.ϵ = nn.Sequential(nn.Linear(pars['width'], 1))
        
    def forward(self, x):
        return torch.hstack((self.n(x), self.u(x), (self.ϵ(x) + 1e-2)**2))

In [12]:
class PINN(nn.Module):
    def __init__(self, pars: dict):
        super().__init__()
        self.pars = pars
        
        self.modules = [nn.Linear(2, self.pars['width'])] # nn.LayerNorm(2)
        for i in range(self.pars['layers'] - 1):
            # self.modules.append(nn.LayerNorm(self.pars['width']))
            self.modules.append(nn.GELU())
            self.modules.append(nn.Linear(self.pars['width'], self.pars['width']))
        
        # self.modules.append(nn.LayerNorm(self.pars['width']))
        self.modules.append(CustomOutputLayer(pars))
        
        self.model = nn.Sequential(*self.modules)
        self.model.to(self.pars['device'])
        
        self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr=self.pars['lr'])
        self.num_params = sum([len(params) for params in [p for p in self.model.parameters()]])
        
        self.epoch = 0
        
        t = np.linspace(self.pars['t_min'], self.pars['t_max'], 100)
        x = np.linspace(self.pars['x_min'], self.pars['x_max'], 100)

        self.eval_t, self.eval_x = np.meshgrid(t, x)
        self.eval_t = torch.Tensor(self.eval_t).reshape(-1, 1).to(self.pars['device'])
        self.eval_x = torch.Tensor(self.eval_x).reshape(-1, 1).to(self.pars['device'])
        
        eval_t_interior, eval_x_interior = np.meshgrid(t[1:-1], x[1:-1])
        eval_t_interior = torch.Tensor(eval_t_interior).reshape(-1, 1).to(self.pars['device'])
        eval_x_interior = torch.Tensor(eval_x_interior).reshape(-1, 1).to(self.pars['device'])
        
        self.eval_X_interior = torch.hstack((eval_t_interior, eval_x_interior))
        self.eval_X_interior.requires_grad_()
        
        eval_t_initial = torch.zeros_like(self.eval_x)
        self.eval_X_initial = torch.hstack((eval_t_initial, self.eval_x))
        self.eval_X_initial.requires_grad_()
        
        eval_t_boundary = torch.Tensor(np.vstack([t, t])).reshape(-1, 1).to(self.pars['device'])
        eval_x_boundary = torch.Tensor(np.vstack([x[0] * np.ones_like(t), x[-1] * np.ones_like(t)])).reshape(-1, 1).to(self.pars['device'])
        
        self.eval_X_boundary = torch.hstack((eval_t_boundary, eval_x_boundary))
        self.eval_X_boundary.requires_grad_()
        
        self.plot_t = self.eval_t.view(100, 100).detach().cpu().numpy()
        self.plot_x = self.eval_x.view(100, 100).detach().cpu().numpy()
        
        self.plot_n_files = []
        self.plot_u_files = []
        self.plot_ϵ_files = []
        
    def __call__(self, X):
        return self.model(X)
        
    def sample_interior_points(self):
        t = torch.empty((self.pars['interior_batch_size'], 1), device=self.pars['device']).uniform_(self.pars['t_min'], self.pars['t_max'])
        x = torch.empty((self.pars['interior_batch_size'], 1), device=self.pars['device']).uniform_(self.pars['x_min'], self.pars['x_max'])
        X_interior = torch.cat((t, x), 1)
        X_interior.requires_grad_()
        
        return X_interior
    
    def sample_initial_points(self):
        t = torch.zeros(self.pars['initial_batch_size'], 1, device=self.pars['device'])
        x = torch.empty((self.pars['initial_batch_size'], 1), device=self.pars['device']).uniform_(self.pars['x_min'], self.pars['x_max'])
        X_initial = torch.cat((t, x), 1)
        X_initial.requires_grad_()
        
        return X_initial
    
    def sample_boundary_points(self):
        options = torch.tensor([self.pars['x_min'], self.pars['x_max']], device=self.pars['device'])
        
        t = torch.empty((self.pars['boundary_batch_size'], 1), device=self.pars['device']).uniform_(self.pars['t_min'], self.pars['t_max'])
        x = options[torch.randint(0, 2, (self.pars['boundary_batch_size'], 1), device=self.pars['device'])]
        X_boundary = torch.cat((t, x), 1)
        X_boundary.requires_grad_()
        
        return X_boundary
    
    def forward(self, X_interior, X_initial, X_boundary):
        # X shape: (batch_size, 2), where 2nd dimension is [t, x]
        # Y shape: (batch_size, 3), where 2nd dimension is [n, u, ϵ]
        
        t_interior = X_interior[:, 0].reshape(-1, 1)
        x_interior = X_interior[:, 1].reshape(-1, 1)
        t_initial = X_initial[:, 0].reshape(-1, 1)
        x_initial = X_initial[:, 1].reshape(-1, 1)
        t_boundary = X_boundary[:, 0].reshape(-1, 1)
        x_boundary = X_boundary[:, 1].reshape(-1, 1)
        
        # forward pass
        Y_interior = self.model(torch.hstack((t_interior, x_interior)))
        Y_initial = self.model(torch.hstack((t_initial, x_initial)))
        Y_boundary = self.model(torch.hstack((t_boundary, x_boundary)))
        
        n_interior = Y_interior[:, 0].reshape(-1, 1)
        u_interior = Y_interior[:, 1].reshape(-1, 1)
        ϵ_interior = Y_interior[:, 2].reshape(-1, 1)
        
        n_initial = Y_initial[:, 0].reshape(-1, 1)
        u_initial = Y_initial[:, 1].reshape(-1, 1)
        ϵ_initial = Y_initial[:, 2].reshape(-1, 1)
        
        n_boundary = Y_boundary[:, 0].reshape(-1, 1)
        u_boundary = Y_boundary[:, 1].reshape(-1, 1)
        ϵ_boundary = Y_boundary[:, 2].reshape(-1, 1)
        
        n_x_interior = grad(n_interior, x_interior, grad_outputs=torch.ones_like(n_interior), retain_graph=True, create_graph=True)[0]
        n_t_interior = grad(n_interior, t_interior, grad_outputs=torch.ones_like(n_interior), retain_graph=True, create_graph=True)[0]
        
        n_xx_interior = grad(n_x_interior, x_interior, grad_outputs=torch.ones_like(n_x_interior), retain_graph=True, create_graph=True)[0]
        
        u_x_interior = grad(u_interior, x_interior, grad_outputs=torch.ones_like(u_interior), retain_graph=True, create_graph=True)[0]
        u_t_interior = grad(u_interior, t_interior, grad_outputs=torch.ones_like(u_interior), retain_graph=True, create_graph=True)[0]
        
        u_xx_interior = grad(u_x_interior, x_interior, grad_outputs=torch.ones_like(u_x_interior), retain_graph=True, create_graph=True)[0]
        
        ϵ_x_interior = grad(ϵ_interior, x_interior, grad_outputs=torch.ones_like(ϵ_interior), retain_graph=True, create_graph=True)[0]
        ϵ_t_interior = grad(ϵ_interior, t_interior, grad_outputs=torch.ones_like(ϵ_interior), retain_graph=True, create_graph=True)[0]
        
        ϵ_x_boundary = grad(ϵ_boundary, x_boundary, grad_outputs=torch.ones_like(ϵ_boundary), retain_graph=True, create_graph=True)[0]
        
        w = compute_w(C_χ, l, ϵ_interior, α, a_u, u_interior)
        
        density_loss = pde_mean_density(x_interior, n_t_interior, n_x_interior, n_xx_interior, ϵ_interior, l, α, D_c).mean()
        vorticity_loss = pde_mean_vorticity(x_interior, ϵ_interior, n_x_interior, u_t_interior, u_xx_interior, l, α, μ_c, w).mean()
        tpe_loss = pde_tpe(x_interior, ϵ_interior, n_x_interior, u_x_interior, ϵ_t_interior, ϵ_x_interior, l, β, Λ, ϵ_c, w).mean()
        interior_loss = (density_loss + vorticity_loss + tpe_loss)/3
        
        mse = nn.MSELoss()
        
        initial_n_loss = mse(n_initial_cond(t_initial, x_initial), n_initial)
        initial_u_loss = mse(u_initial_cond(t_initial, x_initial), u_initial)
        initial_ϵ_loss = mse(ϵ_initial_cond(t_initial, x_initial), ϵ_initial)
        initial_loss = (initial_n_loss + initial_u_loss + initial_ϵ_loss)/3
        
        boundary_n_loss = mse(n_boundary_cond(t_boundary, x_boundary), n_boundary)
        boundary_u_loss = mse(u_boundary_cond(t_boundary, x_boundary), u_boundary)
        boundary_ϵ_loss = mse(ϵ_x_boundary_cond(t_boundary, x_boundary), ϵ_x_boundary)
        boundary_loss = (boundary_n_loss + boundary_u_loss + boundary_ϵ_loss)/3
        total_loss = interior_loss + initial_loss + boundary_loss
        
        return total_loss, density_loss, vorticity_loss, tpe_loss, initial_n_loss, initial_u_loss, initial_ϵ_loss, boundary_n_loss, boundary_u_loss, boundary_ϵ_loss
    
    def train(self):
        cwd = os.getcwd()
        plot_dirs = os.path.join(cwd, f'plots/{self.pars["experiment_name"]}')

        if not os.path.isdir(plot_dirs):
            os.makedirs(plot_dirs)
        
        mlflow.set_experiment(self.pars['experiment_name'])
        mlflow.start_run()
        
        mlflow.log_param("physical_params", physical_params)
        mlflow.log_param("model_params", self.pars)
        
        for epoch in tqdm(range(self.pars['epochs']), position=0, leave=True, desc='Training...'): 
            self.epoch = epoch
            
            # eval
            if epoch % self.pars['eval_interval'] == 0 or epoch == self.pars['epochs'] - 1:
                
                loss, density_loss, vorticity_loss, tpe_loss, initial_n_loss, initial_u_loss, initial_ϵ_loss, boundary_n_loss, boundary_u_loss, boundary_ϵ_loss \
                    = self.forward(self.eval_X_interior, self.eval_X_initial, self.eval_X_boundary)
                
                print()
                print(f'Epoch: {self.epoch}, Loss: {loss.item():,.4e}')
                print(f"density_loss: {density_loss.item():.4e}, vorticity_loss: {vorticity_loss.item():.4e}, tpe_loss: {tpe_loss.item():.4e}")
                print(f"initial_n_loss: {initial_n_loss.item():.4e}, initial_u_loss: {initial_u_loss.item():.4e}, initial_ϵ_loss: {initial_ϵ_loss.item():.4e}")
                print(f"boundary_n_loss: {boundary_n_loss.item():.4e}, boundary_u_loss: {boundary_u_loss.item():.4e}, boundary_ϵ_loss: {boundary_ϵ_loss.item():.4e}")
                
                mlflow.log_metric("total_loss", loss.item(), step=self.epoch)
                mlflow.log_metric("density_loss", density_loss.item(), step=self.epoch)
                mlflow.log_metric("vorticity_loss", vorticity_loss.item(), step=self.epoch)
                mlflow.log_metric("tpe_loss", tpe_loss.item(), step=self.epoch)
                mlflow.log_metric("initial_n_loss", initial_n_loss.item(), step=self.epoch)
                mlflow.log_metric("initial_u_loss", initial_u_loss.item(), step=self.epoch)
                mlflow.log_metric("initial_ϵ_loss", initial_ϵ_loss.item(), step=self.epoch)
                mlflow.log_metric("boundary_n_loss", boundary_n_loss.item(), step=self.epoch)
                mlflow.log_metric("boundary_u_loss", boundary_u_loss.item(), step=self.epoch)
                mlflow.log_metric("boundary_ϵ_loss", boundary_ϵ_loss.item(), step=self.epoch)
                
                mlflow.pytorch.log_model(self.model, f"{self.pars['experiment_name']}_model_epoch_{self.epoch}")
                
                if self.pars['plot_training_outputs']:
                    self.plot_outputs()
            
            # training step
            self.optimizer.zero_grad()
            X_interior = self.sample_interior_points()
            X_initial = self.sample_initial_points()
            X_boundary = self.sample_boundary_points()
            loss, density_loss, vorticity_loss, tpe_loss, initial_n_loss, initial_u_loss, initial_ϵ_loss, boundary_n_loss, boundary_u_loss, boundary_ϵ_loss = self.forward(X_interior, X_initial, X_boundary)
            loss.backward()
            self.optimizer.step()
            
                
            if loss.isnan():
                print(f'Epoch: {self.epoch}, Loss: {loss.item():,.4e}')
                print(f"density_loss: {density_loss.item():.4e}, vorticity_loss: {vorticity_loss.item():.4e}, tpe_loss: {tpe_loss.item():.4e}")
                print(f"initial_n_loss: {initial_n_loss.item():.4e}, initial_u_loss: {initial_u_loss.item():.4e}, initial_ϵ_loss: {initial_ϵ_loss.item():.4e}")
                print(f"boundary_n_loss: {boundary_n_loss.item():.4e}, boundary_u_loss: {boundary_u_loss.item():.4e}, boundary_ϵ_loss: {boundary_ϵ_loss.item():.4e}")
                print("loss is NaN, stopping training...")
                break
            
        mlflow.pytorch.log_model(self.model, f"{self.pars['experiment_name']}_model_final")
        
        self.save_gif()
        
        mlflow.end_run()
    
    def plot_outputs(self):
        with torch.no_grad():
            Y = self(torch.hstack((self.eval_t, self.eval_x)))

        n = Y[:, 0].view(100, 100).detach().cpu().numpy()
        u = Y[:, 1].view(100, 100).detach().cpu().numpy()
        ϵ = Y[:, 2].view(100, 100).detach().cpu().numpy()
        
        self.plot_n(n)
        self.plot_u(u)
        self.plot_ϵ(ϵ)
        
    def plot_n(self, n):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(self.plot_t, self.plot_x, n, cmap='viridis')
        ax.set_xlabel('t')
        ax.set_ylabel('x')
        ax.set_zlabel('n')
        # ax.set_zlim(-g_i-1, 1)
        plt.title(f'Model output n, epoch {self.epoch}')
        filename = f"plots/{self.pars['experiment_name']}/plot_n_epoch_{self.epoch}.png"
        self.plot_n_files.append(filename)
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close()
        
    def plot_u(self, u):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(self.plot_t, self.plot_x, u, cmap='viridis')
        ax.set_xlabel('t')
        ax.set_ylabel('x')
        ax.set_zlabel('u')
        # ax.set_zlim(-0.02, 0.02)
        plt.title(f'Model output u, epoch {self.epoch}')
        filename = f"plots/{self.pars['experiment_name']}/plot_u_epoch_{self.epoch}.png"
        self.plot_u_files.append(filename)
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close()
        
    def plot_ϵ(self, ϵ):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(self.plot_t, self.plot_x, ϵ, cmap='viridis')
        ax.set_xlabel('t')
        ax.set_ylabel('x')
        ax.set_zlabel('ϵ')
        plt.title(f'Model output ϵ, epoch {self.epoch}')
        filename = f"plots/{self.pars['experiment_name']}/plot_ϵ_epoch_{self.epoch}.png"
        self.plot_ϵ_files.append(filename)
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close()
        
    def save_gif(self):
        gif_filename = f"plots/{self.pars['experiment_name']}/plot_n.gif"
        with imageio.get_writer(gif_filename, mode='I') as writer:
            for filename in self.plot_n_files:
                image = imageio.imread(filename)
                writer.append_data(image)
                
        mlflow.log_artifact(gif_filename)
                
        gif_filename = f"plots/{self.pars['experiment_name']}/plot_u.gif"
        with imageio.get_writer(gif_filename, mode='I') as writer:
            for filename in self.plot_u_files:
                image = imageio.imread(filename)
                writer.append_data(image)
                
        mlflow.log_artifact(gif_filename)
                
        gif_filename = f"plots/{self.pars['experiment_name']}/plot_ϵ.gif"
        with imageio.get_writer(gif_filename, mode='I') as writer:
            for filename in self.plot_ϵ_files:
                image = imageio.imread(filename)
                writer.append_data(image)
        
        mlflow.log_artifact(gif_filename)
        

## Experiments

In [13]:
pars = {
    'experiment_name': 'simple_pinn_v1',
    'layers': 4,
    'width': 32,
    'lr': 1e-3,
    'epochs': 100000,
    'eval_interval': 1000,
    'interior_batch_size': 2048,
    'initial_batch_size': 2048,
    'boundary_batch_size': 2048,
    'x_min': 0.0,
    'x_max': 1.0,
    't_min': 0.0,
    't_max': 1,
    'device': 'cuda',
    'plot_training_outputs': True
}
pinn = PINN(pars)


In [14]:
pinn.train()

2024/08/16 16:29:07 INFO mlflow.tracking.fluent: Experiment with name 'simple_pinn_v1' does not exist. Creating a new experiment.


Training...:   0%|          | 0/10000 [00:00<?, ?it/s]


Epoch: 0, Loss: 1.2349e+04
density_loss: 1.3670e-05, vorticity_loss: 5.4540e-05, tpe_loss: 3.7024e+04
initial_n_loss: 9.4595e+00, initial_u_loss: 8.4701e-03, initial_ϵ_loss: 2.5487e-03
boundary_n_loss: 1.3749e+01, boundary_u_loss: 7.8853e-03, boundary_ϵ_loss: 1.0405e-05


2024/08/16 16:29:10 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



Epoch: 500, Loss: 3.5416e+00
density_loss: 2.4428e-03, vorticity_loss: 7.4801e-05, tpe_loss: 6.9820e-02
initial_n_loss: 3.1881e+00, initial_u_loss: 2.4363e-06, initial_ϵ_loss: 3.7844e-06
boundary_n_loss: 7.3642e+00, boundary_u_loss: 4.9974e-06, boundary_ϵ_loss: 3.2073e-07


...

2024/08/16 16:35:41 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



Epoch: 9500, Loss: 6.8040e-05
density_loss: 1.8581e-04, vorticity_loss: 3.0058e-08, tpe_loss: 4.9289e-06
initial_n_loss: 3.0438e-06, initial_u_loss: 1.0673e-07, initial_ϵ_loss: 3.9996e-06
boundary_n_loss: 6.1228e-06, boundary_u_loss: 8.1326e-08, boundary_ϵ_loss: 6.0667e-11


2024/08/16 16:36:02 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



Epoch: 9999, Loss: 4.0586e-05
density_loss: 1.0489e-04, vorticity_loss: 1.8208e-08, tpe_loss: 5.0819e-06
initial_n_loss: 2.2640e-06, initial_u_loss: 1.7921e-07, initial_ϵ_loss: 3.9996e-06
boundary_n_loss: 4.9170e-06, boundary_u_loss: 4.0632e-07, boundary_ϵ_loss: 7.1617e-11


2024/08/16 16:36:28 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/16 16:36:30 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
/tmp/ipykernel_383568/3563034069.py:276: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
/tmp/ipykernel_383568/3563034069.py:284: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
/tmp/ipyke

## Debug

In [ ]:
for epoch in tqdm(range(pinn.pars['epochs']), position=0, leave=True, desc='Training...'): 
    pinn.epoch = epoch
    
    # eval
    if epoch % pinn.pars['eval_interval'] == 0 or epoch == pinn.pars['epochs'] - 1:
        
        loss, density_loss, vorticity_loss, tpe_loss, initial_n_loss, initial_u_loss, initial_ϵ_loss, boundary_n_loss, boundary_u_loss, boundary_ϵ_loss \
            = pinn.forward(pinn.eval_X_interior, pinn.eval_X_initial, pinn.eval_X_boundary)
        
        print()
        print(f'Epoch: {pinn.epoch}, Loss: {loss.item():,.4f}')
        print(f"density_loss: {density_loss.item():.4f}, vorticity_loss: {vorticity_loss.item():.4f}, tpe_loss: {tpe_loss.item():.4f}")
        print(f"initial_n_loss: {initial_n_loss.item():.4f}, initial_u_loss: {initial_u_loss.item():.4f}, initial_ϵ_loss: {initial_ϵ_loss.item():.4f}")
        print(f"boundary_n_loss: {boundary_n_loss.item():.4f}, boundary_u_loss: {boundary_u_loss.item():.4f}, boundary_ϵ_loss: {boundary_ϵ_loss.item():.4f}")
        
        # if pinn.pars['plot_training_outputs']:
        #     pinn.plot_outputs()
    
    # training step
    pinn.optimizer.zero_grad()
    X_interior = pinn.sample_interior_points()
    X_initial = pinn.sample_initial_points()
    X_boundary = pinn.sample_boundary_points()
    loss, density_loss, vorticity_loss, tpe_loss, initial_n_loss, initial_u_loss, initial_ϵ_loss, boundary_n_loss, boundary_u_loss, boundary_ϵ_loss = pinn.forward(X_interior, X_initial, X_boundary)
    
        
    if loss.isnan():
        print(f'Epoch: {pinn.epoch}, Loss: {loss.item():,.4f}')
        print(f"density_loss: {density_loss.item():.4f}, vorticity_loss: {vorticity_loss.item():.4f}, tpe_loss: {tpe_loss.item():.4f}")
        print(f"initial_n_loss: {initial_n_loss.item():.4f}, initial_u_loss: {initial_u_loss.item():.4f}, initial_ϵ_loss: {initial_ϵ_loss.item():.4f}")
        print(f"boundary_n_loss: {boundary_n_loss.item():.4f}, boundary_u_loss: {boundary_u_loss.item():.4f}, boundary_ϵ_loss: {boundary_ϵ_loss.item():.4f}")
        print("loss is NaN, stopping training...")
        break
    
    else:
        loss.backward()
        pinn.optimizer.step()

In [ ]:
x_interior = X_interior[:, 1].reshape(-1, 1)
t_interior = X_interior[:, 0].reshape(-1, 1)

Y_interior = pinn(torch.hstack((t_interior, x_interior)))

In [ ]:


n_interior = Y_interior[:, 0].reshape(-1, 1)
u_interior = Y_interior[:, 1].reshape(-1, 1)
ϵ_interior = Y_interior[:, 2].reshape(-1, 1)

w = compute_w(C_χ, l, ϵ_interior, α, a_u, u_interior)

n_x_interior = grad(n_interior, x_interior, grad_outputs=torch.ones_like(n_interior), retain_graph=True, create_graph=True)[0]
u_x_interior = grad(u_interior, x_interior, grad_outputs=torch.ones_like(u_interior), retain_graph=True, create_graph=True)[0]
ϵ_x_interior = grad(ϵ_interior, x_interior, grad_outputs=torch.ones_like(ϵ_interior), retain_graph=True, create_graph=True)[0]
ϵ_t_interior = grad(ϵ_interior, t_interior, grad_outputs=torch.ones_like(ϵ_interior), retain_graph=True, create_graph=True)[0]

In [ ]:
tpe = pde_tpe(x_interior, ϵ_interior, n_x_interior, u_x_interior, ϵ_t_interior, ϵ_x_interior, l, β, Λ, ϵ_c, w)

In [ ]:
x_interior[336], ϵ_interior[336], n_x_interior[336], u_x_interior[336], ϵ_x_interior[336], ϵ_t_interior[336], tpe[336]

In [ ]:
intermediate = l**2 * torch.sqrt(ϵ_interior) * ϵ_x_interior
intermediate.isnan().sum()

In [ ]:
ϵ_interior[336]

In [ ]:
(ϵ_interior**(-0.5))[336]

In [ ]:
(ϵ_interior**(-0.5)*ϵ_x_interior).isnan().sum()

In [ ]:
test = grad(ϵ_x_interior, x_interior, grad_outputs=torch.ones_like(ϵ_x_interior), retain_graph=True, create_graph=True)[0]

In [ ]:
test.isnan().sum()

In [ ]:
intermediate_x = grad(intermediate, x_interior, grad_outputs=torch.ones_like(intermediate), retain_graph=True, create_graph=True)[0]
intermediate_x.isnan().sum()

In [ ]:
torch.autograd.detect_anomaly(check_nan=True)

In [ ]:
for i in range(tpe.shape[0]):
    if tpe[i].isnan():
        print(i)
        break

In [ ]:
for p in pinn.parameters():
    print(p)

In [16]:
X_interior = pinn.sample_interior_points()
X_initial = pinn.sample_initial_points()
X_boundary = pinn.sample_boundary_points()

t_interior = X_interior[:, 0].reshape(-1, 1)
x_interior = X_interior[:, 1].reshape(-1, 1)
t_initial = X_initial[:, 0].reshape(-1, 1)
x_initial = X_initial[:, 1].reshape(-1, 1)
t_boundary = X_boundary[:, 0].reshape(-1, 1)
x_boundary = X_boundary[:, 1].reshape(-1, 1)

# forward pass
Y_interior = pinn.model(torch.hstack((t_interior, x_interior)))
Y_initial = pinn.model(torch.hstack((t_initial, x_initial)))
Y_boundary = pinn.model(torch.hstack((t_boundary, x_boundary)))

In [17]:
print(f"X_interior: {X_interior}")
print(f"X_initial: {X_initial}")
print(f"X_boundary: {X_boundary}")
print(f"t_interior: {t_interior}")
print(f"x_interior: {x_interior}")
print(f"t_initial: {t_initial}")
print(f"x_initial: {x_initial}")
print(f"t_boundary: {t_boundary}")
print(f"x_boundary: {x_boundary}")
print(f"Y_interior: {Y_interior}")
print(f"Y_initial: {Y_initial}")
print(f"Y_boundary: {Y_boundary}")

X_interior: tensor([[0.3982, 0.4681],
        [0.9412, 0.4704],
        [0.1411, 0.3022],
        ...,
        [0.2676, 0.2029],
        [0.2089, 0.4961],
        [0.6565, 0.8489]], device='cuda:0', requires_grad=True)
X_initial: tensor([[0.0000, 0.0455],
        [0.0000, 0.5064],
        [0.0000, 0.7834],
        ...,
        [0.0000, 0.9536],
        [0.0000, 0.7343],
        [0.0000, 0.8111]], device='cuda:0', requires_grad=True)
X_boundary: tensor([[0.8714, 0.0000],
        [0.4738, 0.0000],
        [0.7147, 1.0000],
        ...,
        [0.9609, 0.0000],
        [0.4535, 0.0000],
        [0.5562, 1.0000]], device='cuda:0', requires_grad=True)
t_interior: tensor([[0.3982],
        [0.9412],
        [0.1411],
        ...,
        [0.2676],
        [0.2089],
        [0.6565]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
x_interior: tensor([[0.4681],
        [0.4704],
        [0.3022],
        ...,
        [0.2029],
        [0.4961],
        [0.8489]], device='cuda:0', grad_fn=<

In [18]:
n_interior = Y_interior[:, 0].reshape(-1, 1)
u_interior = Y_interior[:, 1].reshape(-1, 1)
ϵ_interior = Y_interior[:, 2].reshape(-1, 1)

n_initial = Y_initial[:, 0].reshape(-1, 1)
u_initial = Y_initial[:, 1].reshape(-1, 1)
ϵ_initial = Y_initial[:, 2].reshape(-1, 1)

n_boundary = Y_boundary[:, 0].reshape(-1, 1)
u_boundary = Y_boundary[:, 1].reshape(-1, 1)
ϵ_boundary = Y_boundary[:, 2].reshape(-1, 1)

In [19]:
print(f"n_interior: {n_interior}")
print(f"u_interior: {u_interior}")
print(f"ϵ_interior: {ϵ_interior}")
print(f"n_initial: {n_initial}")
print(f"u_initial: {u_initial}")
print(f"ϵ_initial: {ϵ_initial}")
print(f"n_boundary: {n_boundary}")
print(f"u_boundary: {u_boundary}")
print(f"ϵ_boundary: {ϵ_boundary}")

n_interior: tensor([[-2.3859],
        [-2.4002],
        [-1.5409],
        ...,
        [-1.0332],
        [-2.5295],
        [-4.3297]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
u_interior: tensor([[-0.0005],
        [-0.0005],
        [-0.0004],
        ...,
        [-0.0003],
        [-0.0005],
        [-0.0005]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
ϵ_interior: tensor([[8.1106e-11],
        [3.4494e-10],
        [5.9049e-09],
        ...,
        [4.9879e-08],
        [2.8602e-08],
        [2.3885e-08]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
n_initial: tensor([[-0.2341],
        [-2.5828],
        [-3.9937],
        ...,
        [-4.8614],
        [-3.7437],
        [-4.1352]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
u_initial: tensor([[-2.3770e-05],
        [-6.1229e-04],
        [-8.9815e-04],
        ...,
        [-1.0225e-03],
        [-8.5467e-04],
        [-9.2126e-04]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
ϵ_initial:

In [20]:
n_x_interior = grad(n_interior, x_interior, grad_outputs=torch.ones_like(n_interior), retain_graph=True, create_graph=True)[0]
n_t_interior = grad(n_interior, t_interior, grad_outputs=torch.ones_like(n_interior), retain_graph=True, create_graph=True)[0]

n_xx_interior = grad(n_x_interior, x_interior, grad_outputs=torch.ones_like(n_x_interior), retain_graph=True, create_graph=True)[0]

u_x_interior = grad(u_interior, x_interior, grad_outputs=torch.ones_like(u_interior), retain_graph=True, create_graph=True)[0]
u_t_interior = grad(u_interior, t_interior, grad_outputs=torch.ones_like(u_interior), retain_graph=True, create_graph=True)[0]

u_xx_interior = grad(u_x_interior, x_interior, grad_outputs=torch.ones_like(u_x_interior), retain_graph=True, create_graph=True)[0]

ϵ_x_interior = grad(ϵ_interior, x_interior, grad_outputs=torch.ones_like(ϵ_interior), retain_graph=True, create_graph=True)[0]
ϵ_t_interior = grad(ϵ_interior, t_interior, grad_outputs=torch.ones_like(ϵ_interior), retain_graph=True, create_graph=True)[0]

ϵ_x_boundary = grad(ϵ_boundary, x_boundary, grad_outputs=torch.ones_like(ϵ_boundary), retain_graph=True, create_graph=True)[0]

In [21]:
print(f"n_x_interior: {n_x_interior}")
print(f"n_t_interior: {n_t_interior}")
print(f"n_xx_interior: {n_xx_interior}")
print(f"u_x_interior: {u_x_interior}")
print(f"u_t_interior: {u_t_interior}")
print(f"u_xx_interior: {u_xx_interior}")
print(f"ϵ_x_interior: {ϵ_x_interior}")
print(f"ϵ_t_interior: {ϵ_t_interior}")
print(f"ϵ_x_boundary: {ϵ_x_boundary}")

n_x_interior: tensor([[-5.1053],
        [-5.0974],
        [-5.1003],
        ...,
        [-5.1029],
        [-5.1014],
        [-5.1033]], device='cuda:0', grad_fn=<SliceBackward0>)
n_t_interior: tensor([[ 0.0019],
        [-0.0118],
        [ 0.0110],
        ...,
        [ 0.0089],
        [ 0.0041],
        [ 0.0008]], device='cuda:0', grad_fn=<SliceBackward0>)
n_xx_interior: tensor([[ 0.0075],
        [-0.0003],
        [ 0.0011],
        ...,
        [ 0.0053],
        [ 0.0095],
        [-0.0059]], device='cuda:0', grad_fn=<SliceBackward0>)
u_x_interior: tensor([[-5.0228e-04],
        [ 3.8102e-04],
        [-1.0184e-03],
        ...,
        [-7.3032e-04],
        [-8.2854e-04],
        [ 8.1598e-05]], device='cuda:0', grad_fn=<SliceBackward0>)
u_t_interior: tensor([[ 1.4177e-04],
        [-7.5041e-05],
        [-7.3105e-05],
        ...,
        [-3.1812e-04],
        [ 2.7365e-04],
        [ 5.6969e-04]], device='cuda:0', grad_fn=<SliceBackward0>)
u_xx_interior: tensor([[ 3

In [ ]:
w = compute_w(C_χ, l, ϵ_interior, α, a_u, u_interior)

density_loss = pde_mean_density(x_interior, n_t_interior, n_x_interior, n_xx_interior, ϵ_interior, l, α, D_c)
vorticity_loss = pde_mean_vorticity(x_interior, ϵ_interior, n_x_interior, u_t_interior, u_xx_interior, l, α, μ_c, w)
tpe_loss = pde_tpe(x_interior, ϵ_interior, n_x_interior, u_x_interior, ϵ_t_interior, ϵ_x_interior, l, β, Λ, ϵ_c, w)
interior_loss = (density_loss + vorticity_loss + tpe_loss).mean()

mse = nn.MSELoss()

initial_n_loss = mse(n_initial_cond(t_initial), n_initial)
initial_u_loss = mse(u_initial_cond(t_initial), u_initial)
initial_ϵ_loss = mse(ϵ_initial_cond(t_initial), ϵ_initial)
initial_loss = (initial_n_loss + initial_u_loss + initial_ϵ_loss)

boundary_n_loss = mse(n_boundary_cond(t_boundary, x_boundary), n_boundary)
boundary_u_loss = mse(u_boundary_cond(t_boundary, x_boundary), u_boundary)
boundary_ϵ_loss = mse(ϵ_x_boundary_cond(t_boundary, x_boundary), ϵ_x_boundary)
boundary_loss = (boundary_n_loss + boundary_u_loss + boundary_ϵ_loss)
total_loss = interior_loss + initial_loss + boundary_loss

In [ ]:
print(f"w: {w}")
print(f"density_loss: {density_loss.mean().item()}, vorticity_loss: {vorticity_loss.mean().item()}, tpe_loss: {tpe_loss.mean().item()}")
print(f"initial_n_loss: {initial_n_loss.item()}, initial_u_loss: {initial_u_loss.item()}, initial_ϵ_loss: {initial_ϵ_loss.item()}")
print(f"boundary_n_loss: {boundary_n_loss.item()}, boundary_u_loss: {boundary_u_loss.item()}, boundary_ϵ_loss: {boundary_ϵ_loss.item()}")
print(f"total_loss: {total_loss.item()}")
print(f"interior_loss: {interior_loss.item()}")
print(f"initial_loss: {initial_loss.item()}")
print(f"boundary_loss: {boundary_loss.item()}")

In [ ]:
n_boundary, n_boundary_cond(t_boundary, x_boundary)

In [ ]:
mse(n_boundary_cond(t_boundary, x_boundary), n_boundary)

In [ ]:
intermediate = l**2 * torch.sqrt(ϵ_interior) * ϵ_x_interior
intermediate_x = grad(intermediate, x_interior, grad_outputs=torch.ones_like(intermediate), retain_graph=True, create_graph=True)[0]

tpe_loss = torch.abs(ϵ_t_interior - β * intermediate_x - Λ * (w * (n_x_interior - u_x_interior)**2 - ϵ_interior**(3/2) / ϵ_c**0.5 + ϵ_interior))

In [ ]:
(w * (n_x_interior - u_x_interior)**2 - ϵ_interior**(3/2) / ϵ_c**0.5 + ϵ_interior)*4000

In [ ]:
ϵ_interior

In [ ]:
ϵ_interior**1.5

In [ ]:
(-0.4184)**1.5